<a href="https://colab.research.google.com/github/Ahmetyldrr/LLM-Models-Pratic/blob/main/Chapter2_LLM_de_Anlam_Arayisi_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LLM de anlam arayısı Part 1
https://colab.research.google.com/drive/1cMkV8lBH3gW1Xfd92rz3EikYRMfpU0Jf#scrollTo=0VmpbA-0Eg-t

# BiEncoder

In [ ]:
!pip install dotenv

In [ ]:
!pip install pinecone-client openai sentence-transformers tiktoken datasets pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from openai import OpenAI
from datetime import datetime
import hashlib
import re
from dotenv import load_dotenv
import os
from sentence_transformers import CrossEncoder
from tqdm import tqdm
import numpy as np
from torch import nn
import logging
import pinecone
from pinecone import ServerlessSpec
from pinecone import Pinecone
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
load_dotenv("/content/drive/MyDrive/.env")
openai_key = os.getenv("OPENAI_API_KEY")
pinecone_key =  os.getenv("PINECONE_API_KEY")

'pcsk_2VS8wV_5TsxayCuNMcLoGjTAw3Adq3darUoHy8YyF2ij6AaaiYRwF8kg7AXRwvAPPt3R7k'

In [ ]:
client = OpenAI(api_key=openai_key)
pc = Pinecone(api_key=pinecone_key)
pc

In [ ]:
INDEX_NAME = 'semantic-search-test'
NAMESPACE = 'default'
ENGINE = 'text-embedding-3-large'  # has vector size 3072

In [ ]:
index = pc.Index(name=INDEX_NAME)

In [ ]:
def my_hash(s):
    # Return the MD5 hash of the input string as a hexadecimal string
    return hashlib.md5(s.encode()).hexdigest()

In [ ]:
# helper functions to get lists of embeddings from the OpenAI API
def get_embeddings(texts, engine=ENGINE):
    response = client.embeddings.create(
        input=texts,
        model=engine
    )

    return [d.embedding for d in list(response.data)]

def get_embedding(text, engine=ENGINE):
    return get_embeddings([text], engine)[0]


In [ ]:
from datetime import datetime

def prepare_for_pinecone(texts):
    now_str = datetime.utcnow().isoformat()  # Tarihi string formatına çeviriyoruz
    embeddings = get_embeddings(texts)
    data = []

    for text, embedding in zip(texts, embeddings):
        data.append((
            my_hash(text),    # Benzersiz ID
            embedding,        # Embedding vektörü
            {
                "text": text,
                "date_uploaded": now_str  # datetime yerine string kullanıyoruz
            }
        ))

    return data


In [ ]:
def upload_texts_to_pinecone(texts, namespace=NAMESPACE, batch_size=None, show_progress_bar=False):
    # Call the prepare_for_pinecone function to prepare the input texts for indexing
    total_upserted = 0
    if not batch_size:
        batch_size = len(texts)

    _range = range(0, len(texts), batch_size)
    for i in tqdm(_range) if show_progress_bar else _range:
        batch = texts[i: i + batch_size]
        prepared_texts = prepare_for_pinecone(batch)

        # Use the upsert() method of the index object to upload the prepared texts to Pinecone
        total_upserted += index.upsert(
            vectors=prepared_texts,
            namespace=namespace
        )['upserted_count']


    return total_upserted


In [ ]:
def query_from_pinecone(query, top_k=3, include_metadata=True):
    # get embedding from THE SAME embedder as the documents
    query_embedding = get_embedding(query, engine=ENGINE)

    return index.query(
      vector=query_embedding,
      top_k=top_k,
      namespace=NAMESPACE,
      include_metadata=include_metadata   # gets the metadata (dates, text, etc)
    ).get('matches')

In [ ]:
def delete_texts_from_pinecone(texts, namespace=NAMESPACE):
    # Compute the hash (id) for each text
    hashes = [hashlib.md5(text.encode()).hexdigest() for text in texts]

    # The ids parameter is used to specify the list of IDs (hashes) to delete
    return index.delete(ids=hashes, namespace=namespace)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("xtreme", "MLQA.en.en")

# rename test -> train and val -> test (as we will use it in later in this chapter)
dataset['train'] = dataset['test']
dataset['test'] = dataset['validation']
del dataset['validation']

dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/7.49M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/724k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/11590 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1148 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1148
    })
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11590
    })
})

In [ ]:
dataset['train'][0], dataset['train'][1]

({'id': 'a4968ca8a18de16aa3859be760e43dbd3af3fce9',
  'title': 'Area 51',
  'context': 'In 1994, five unnamed civilian contractors and the widows of contractors Walter Kasza and Robert Frost sued the USAF and the United States Environmental Protection Agency. Their suit, in which they were represented by George Washington University law professor Jonathan Turley, alleged they had been present when large quantities of unknown chemicals had been burned in open pits and trenches at Groom. Biopsies taken from the complainants were analyzed by Rutgers University biochemists, who found high levels of dioxin, dibenzofuran, and trichloroethylene in their body fat. The complainants alleged they had sustained skin, liver, and respiratory injuries due to their work at Groom, and that this had contributed to the deaths of Frost and Kasza. The suit sought compensation for the injuries they had sustained, claiming the USAF had illegally handled toxic materials, and that the EPA had failed in its dut

In [ ]:
unique_passages = list(set(dataset['test']['context']))
for idx in tqdm(range(0, len(unique_passages), 32)):
    passages = unique_passages[idx:idx + 32]
    upload_texts_to_pinecone(passages)

100%|██████████| 31/31 [02:01<00:00,  3.93s/it]


In [ ]:
len(unique_passages)

978

In [ ]:
index.describe_index_stats()

{'dimension': 3072,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 8}, 'default': {'vector_count': 978}},
 'total_vector_count': 986,
 'vector_type': 'dense'}

In [ ]:
dataset['test'][0]

{'id': '569666f4dc3983dab5624e989212c1d9d0cd1798',
 'title': 'Pappataci fever',
 'context': 'Pappataci fever is prevalent in the subtropical zone of the Eastern Hemisphere between 20°N and 45°N, particularly in Southern Europe, North Africa, the Balkans, Eastern Mediterranean, Iraq, Iran, Pakistan, Afghanistan and India.The disease is transmitted by the bites of phlebotomine sandflies of the Genus Phlebotomus, in particular, Phlebotomus papatasi, Phlebotomus perniciosus and Phlebotomus perfiliewi. The sandfly becomes infected when biting an infected human in the period between 48 hours before the onset of fever and 24 hours after the end of the fever, and remains infected for its lifetime. Besides this horizontal virus transmission from man to sandfly, the virus can be transmitted in insects transovarially, from an infected female sandfly to its offspring.Pappataci fever is seldom recognised in endemic populations because it is mixed with other febrile illnesses of childhood, but it is

In [ ]:
query_from_pinecone('Does an infection for Sandflies go away over time?')

[{'id': '2f90090e21f19450887d5f3ff781e541',
  'metadata': {'date_uploaded': '2025-03-25T10:07:05.140762',
               'text': 'Pappataci fever is prevalent in the subtropical zone of '
                       'the Eastern Hemisphere between 20°N and 45°N, '
                       'particularly in Southern Europe, North Africa, the '
                       'Balkans, Eastern Mediterranean, Iraq, Iran, Pakistan, '
                       'Afghanistan and India.The disease is transmitted by the '
                       'bites of phlebotomine sandflies of the Genus '
                       'Phlebotomus, in particular, Phlebotomus papatasi, '
                       'Phlebotomus perniciosus and Phlebotomus perfiliewi. The '
                       'sandfly becomes infected when biting an infected human '
                       'in the period between 48 hours before the onset of '
                       'fever and 24 hours after the end of the fever, and '
                       'remains infec

In [ ]:
query_from_pinecone('what is artifical intelligence')

[{'id': '2f74c7188df798a7b01d1d21a59ed44d',
  'metadata': {'date_uploaded': '2025-03-25T10:06:29.940194',
               'text': 'In October 2015, the original AlphaGo became the first '
                       'computer Go program to beat a human professional Go '
                       'player without handicaps on a full-sized 19×19 board. '
                       'In March 2016, it beat Lee Sedol in a five-game match, '
                       'the first time a computer Go program has beaten a 9-dan '
                       'professional without handicaps. Although it lost to Lee '
                       'Sedol in the fourth game, Lee resigned in the final '
                       'game, giving a final score of 4 games to 1 in favour of '
                       'AlphaGo. In recognition of the victory, AlphaGo was '
                       'awarded an honorary 9-dan by the Korea Baduk '
                       'Association. The lead up and the challenge match with '
                     

# CrossEncoder

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder
import numpy as np
from torch import nn
from copy import copy

### Test Sürüşleri

In [ ]:
from copy import copy

def get_results_from_pinecone(query, top_k=3, re_rank_model=None, verbose=True, correct_hash=None):

    results_from_pinecone = query_from_pinecone(query, top_k=top_k)

    if not results_from_pinecone:
        return []

    if verbose:
        print("Query:", query)


    final_results = []

    retrieved_correct_position, reranked_correct_position = None, None
    for idx, result_from_pinecone in enumerate(results_from_pinecone):
        if correct_hash and result_from_pinecone['id'] == correct_hash:
            retrieved_correct_position = idx

    if re_rank_model is not None:
        if verbose:
            print('Document ID (Hash)\t\tRetrieval Score\tCE Score\tText')

        sentence_combinations = [[query, result_from_pinecone['metadata']['text']] for result_from_pinecone in results_from_pinecone]

        # Compute the similarity scores for these combinations
        similarity_scores = re_rank_model.predict(sentence_combinations, activation_fct=nn.Sigmoid())

        # Sort the scores in decreasing order
        sim_scores_argsort = list(reversed(np.argsort(similarity_scores)))
        sim_scores_sort = list(reversed(np.sort(similarity_scores)))
        top_re_rank_score = sim_scores_sort[0]

        # Print the scores
        # print(list(zip(sim_scores_argsort, sim_scores_sort)))
        for idx, _ in enumerate(sim_scores_argsort):
            result_from_pinecone = results_from_pinecone[_]
            if correct_hash and retrieved_correct_position == _:
                reranked_correct_position = idx
            final_results.append({'score': similarity_scores[idx], 'id': result_from_pinecone['id'], 'metadata': result_from_pinecone['metadata']})
            if verbose:
                print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{similarity_scores[idx]:.6f}\t{result_from_pinecone['metadata']['text'][:50]}")
        return {'final_results': final_results, 'retrieved_correct_position': retrieved_correct_position, 'reranked_correct_position': reranked_correct_position, 'results_from_pinecone': results_from_pinecone, 'top_re_rank_score': top_re_rank_score}

    if verbose:
        print('Document ID (Hash)\t\tRetrieval Score\tText')
    for result_from_pinecone in results_from_pinecone:
        final_results.append(result_from_pinecone)
        if verbose:
            print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")

    return {'final_results': final_results, 'retrieved_correct_position': retrieved_correct_position, 'reranked_correct_position': reranked_correct_position}

In [ ]:
query = 'Does an infection for Sandflies go away over time?'

In [ ]:
 top_k=10
 results_from_pinecone = query_from_pinecone(query, top_k=top_k)

 for idx in  results_from_pinecone:
    print(idx["id"])




2f90090e21f19450887d5f3ff781e541
00661b04eb84a4664717245513ea30cd
24d322a7ce10e9e2fa12fcf54f50c651
bc980fa5c763b4af5d02701b42ee1604
e7c15a5e2e681d86950cf6115d8de30d
4b6ec8626208d2aadde32fd069c58ae2
4dc963bb7552b8109e729cc174896cf0
20d2ff7276da85340534dabf110da251
321dff23b195957509e74a7eef37235d
39a4e46adb19c2f61c699d8db222d2dc


In [ ]:
    final_results = []

    retrieved_correct_position, reranked_correct_position = None, None
    for idx, result_from_pinecone in enumerate(results_from_pinecone):
        if result_from_pinecone['id'] == "4dc963bb7552b8109e729cc174896cf0":
            retrieved_correct_position = idx

In [ ]:
retrieved_correct_position

6

In [ ]:
sentence_combinations = [[query, result_from_pinecone['metadata']['text']] for result_from_pinecone in results_from_pinecone]
sentence_combinations

[['Does an infection for Sandflies go away over time?',
  'Pappataci fever is prevalent in the subtropical zone of the Eastern Hemisphere between 20°N and 45°N, particularly in Southern Europe, North Africa, the Balkans, Eastern Mediterranean, Iraq, Iran, Pakistan, Afghanistan and India.The disease is transmitted by the bites of phlebotomine sandflies of the Genus Phlebotomus, in particular, Phlebotomus papatasi, Phlebotomus perniciosus and Phlebotomus perfiliewi. The sandfly becomes infected when biting an infected human in the period between 48 hours before the onset of fever and 24 hours after the end of the fever, and remains infected for its lifetime. Besides this horizontal virus transmission from man to sandfly, the virus can be transmitted in insects transovarially, from an infected female sandfly to its offspring.Pappataci fever is seldom recognised in endemic populations because it is mixed with other febrile illnesses of childhood, but it is more well-known among immigrants 

In [ ]:
len(sentence_combinations)

10

In [ ]:
sentence_combinations[0]

['Does an infection for Sandflies go away over time?',
 'Pappataci fever is prevalent in the subtropical zone of the Eastern Hemisphere between 20°N and 45°N, particularly in Southern Europe, North Africa, the Balkans, Eastern Mediterranean, Iraq, Iran, Pakistan, Afghanistan and India.The disease is transmitted by the bites of phlebotomine sandflies of the Genus Phlebotomus, in particular, Phlebotomus papatasi, Phlebotomus perniciosus and Phlebotomus perfiliewi. The sandfly becomes infected when biting an infected human in the period between 48 hours before the onset of fever and 24 hours after the end of the fever, and remains infected for its lifetime. Besides this horizontal virus transmission from man to sandfly, the virus can be transmitted in insects transovarially, from an infected female sandfly to its offspring.Pappataci fever is seldom recognised in endemic populations because it is mixed with other febrile illnesses of childhood, but it is more well-known among immigrants an

In [ ]:
retrieved_correct_position, reranked_correct_position = None, None
for idx, result_from_pinecone in enumerate(results_from_pinecone):
        if "4dc963bb7552b8109e729cc174896cf0" and result_from_pinecone['id'] == "4dc963bb7552b8109e729cc174896cf0":
            retrieved_correct_position = idx

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', num_labels=1)
cross_encoder

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
q_to_hash = {data['question']: my_hash(data['context']) for data in dataset['test']}
q_to_hash

{'Does an infection for Sandflies go away over time?': '2f90090e21f19450887d5f3ff781e541',
 'What is the name of a type of string that constitutes parts of celery?': '49201636ad4102735125e146c0dbafa4',
 'What are made up almost entirely of parenchyma cells?': '80d92494d2b06f341842f1855d2938cf',
 'What is the leptome?': 'e3fd54f33b021ea3cf88b438fefcada7',
 'what are the resultant of plant cell differentiation?': '37ed40c20d4e1b9cc8d6cc27a4d0fff3',
 'Epidermal cells can do what?': 'c72c2f3c9b9d3353fb2667022237d93e',
 'What kind of plants possess xylem?': '4866ebb2e2b91045d75773b889fef4f9',
 'Fibers, parenchyma cella and which other vessels compose Xylem?': '4866ebb2e2b91045d75773b889fef4f9',
 'In what area was his Diploma?': 'f71b14d780dfdb2aef9da1eb0dd22ed5',
 'What school did Poe attend?': '841d6bcca8d11ce478202709428ff4be',
 'What was the originally planned name for this work?': 'd5a5d91712ad00c9e346c168d6096a53',
 'Who did Poe marry eventually?': '89d8ec718c9bc730f16e7146b47b33ed',
 

In [ ]:
unique_inputs = list(set(dataset['test']['question']))
len(unique_inputs)

1148

In [ ]:
query = unique_inputs[0]
print(query)

for t in dataset['test']:
    if t['question'] == query:
        print(t['context'])

Who was a member of Def Leppard?
Elliott proposed the name "Deaf Leopard" which was originally a band name he thought up while writing reviews for imaginary rock bands in his English class. At Kenning's suggestion, the spelling was slightly modified in order to make the name seem less like that of a punk band. In January 1978, Steve Clark joined the band.  According to Elliott, he successfully auditioned for the band by playing Lynyrd Skynyrd's "Free Bird" in its entirety.In November, just prior to recording sessions for what would be a three-song release known as The Def Leppard E.P., Kenning abruptly left the band; he would later form the band Cairo. He was replaced for those sessions by Frank Noon. By the end of the month, Rick Allen, then only 15 years old, had joined the band as its full-time drummer. Sales of the EP soared after the track "Getcha Rocks Off" was given extensive airtime by renowned BBC Radio 1 DJ John Peel, considered at the time to be a champion of punk rock and n

In [ ]:
query_result = get_results_from_pinecone(
    query,
    top_k=2, # grab 2 results
    re_rank_model=cross_encoder,
    correct_hash=q_to_hash[query],
    verbose=False
    )

query_result['retrieved_correct_position'], query_result['reranked_correct_position']

(0, 0)

In [ ]:
query_result

{'final_results': [{'score': np.float32(0.9867491),
   'id': 'ff83dc24ad12172e9564b31673baecfd',
   'metadata': {'date_uploaded': '2025-03-25T10:07:49.956540',
    'text': 'Elliott proposed the name "Deaf Leopard" which was originally a band name he thought up while writing reviews for imaginary rock bands in his English class. At Kenning\'s suggestion, the spelling was slightly modified in order to make the name seem less like that of a punk band. In January 1978, Steve Clark joined the band.  According to Elliott, he successfully auditioned for the band by playing Lynyrd Skynyrd\'s "Free Bird" in its entirety.In November, just prior to recording sessions for what would be a three-song release known as The Def Leppard E.P., Kenning abruptly left the band; he would later form the band Cairo. He was replaced for those sessions by Frank Noon. By the end of the month, Rick Allen, then only 15 years old, had joined the band as its full-time drummer. Sales of the EP soared after the track "

In [ ]:
test_sample = dataset['test']

In [ ]:
TOP_K=50

In [ ]:
logger.setLevel(logging.CRITICAL)

predictions = []

for question in tqdm(test_sample['question']):
    r = get_results_from_pinecone(
        question, top_k=TOP_K, re_rank_model=cross_encoder, correct_hash=q_to_hash[question],
        verbose=False
        )

    r['retrieved_correct_position'], r['reranked_correct_position']
    predictions.append(r)
    if len(predictions) % 100 == 0:
        retrieved_accuracy = sum([_['retrieved_correct_position'] == 0 for _ in predictions])/len(predictions)
        re_ranked_accuracy = sum([_['reranked_correct_position'] == 0 for _ in predictions])/len(predictions)

        print(f'Accuracy without re-ranking: {retrieved_accuracy}')
        print(f'Accuracy with re-ranking: {re_ranked_accuracy}')

  9%|▊         | 100/1148 [03:09<31:46,  1.82s/it]

Accuracy without re-ranking: 0.78
Accuracy with re-ranking: 0.84


 17%|█▋        | 200/1148 [06:17<29:06,  1.84s/it]

Accuracy without re-ranking: 0.765
Accuracy with re-ranking: 0.835


 26%|██▌       | 300/1148 [09:29<25:47,  1.83s/it]

...

 87%|████████▋ | 1000/1148 [30:59<04:44,  1.92s/it]

Accuracy without re-ranking: 0.742
Accuracy with re-ranking: 0.82


 96%|█████████▌| 1100/1148 [34:02<01:26,  1.81s/it]

Accuracy without re-ranking: 0.7518181818181818
Accuracy with re-ranking: 0.8281818181818181


100%|██████████| 1148/1148 [35:31<00:00,  1.86s/it]


In [ ]:
retrieved_accuracy = sum([_['retrieved_correct_position'] == 0 for _ in predictions])/len(predictions)
re_ranked_accuracy = sum([_['reranked_correct_position'] == 0 for _ in predictions])/len(predictions)

print(f'Accuracy without re-ranking: {retrieved_accuracy}')
print(f'Accuracy with re-ranking: {re_ranked_accuracy}')

Accuracy without re-ranking: 0.7552264808362369
Accuracy with re-ranking: 0.8336236933797909


In [ ]:
import pandas as pd

predictions_df = pd.DataFrame(predictions)
predictions_df.head()

,final_results,retrieved_correct_position,reranked_correct_position,results_from_pinecone,top_re_rank_score
0,"[{'score': 0.29227442, 'id': '2f90090e21f19450...",0.0,0.0,"[{'id': '2f90090e21f19450887d5f3ff781e541', '...",0.292274
1,"[{'score': 0.17795162, 'id': '49201636ad410273...",0.0,0.0,"[{'id': '49201636ad4102735125e146c0dbafa4', '...",0.177952
2,"[{'score': 0.9939328, 'id': '80d92494d2b06f341...",0.0,0.0,"[{'id': '80d92494d2b06f341842f1855d2938cf', '...",0.993933
3,"[{'score': 0.16567454, 'id': '99247f2042d1ed0b...",0.0,2.0,"[{'id': 'e3fd54f33b021ea3cf88b438fefcada7', '...",0.848871
4,"[{'score': 0.8213388, 'id': '37ed40c20d4e1b9cc...",0.0,0.0,"[{'id': '37ed40c20d4e1b9cc8d6cc27a4d0fff3', '...",0.821339


In [ ]:
predictions_df[['retrieved_correct_position', 'reranked_correct_position']].mean()  # lower is better

,0
retrieved_correct_position,1.304813
reranked_correct_position,0.680036


In [ ]:
# do recall @ 1, 3, 5, 10, etc
X = [1, 3, 5, 10, 25, 50]
OPENAI_RETRIEVAL = []
OLD_CROSS_ENCODER = []

for k in X:
    embedding_only_recall = predictions_df[predictions_df['retrieved_correct_position'] < k].shape[0]
    reranked_recall = predictions_df[predictions_df['reranked_correct_position'] < k].shape[0]
    OPENAI_RETRIEVAL.append(embedding_only_recall / predictions_df.shape[0])
    OLD_CROSS_ENCODER.append(reranked_recall / predictions_df.shape[0])
    print(k, embedding_only_recall, reranked_recall)

1 867 957
3 1009 1057
5 1048 1076
10 1073 1094
25 1108 1120
50 1122 1122


#Fine-tuning